# Optimize Machine Learning - Bank Marketing Predictor

## Training Setup

### Initialize Workspace

In [1]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(f"Azure Machine Learning Workspace Loaded Successfully\n"
      f"-----------------------------------------------------------\n"
      f"Workspace Name     : {ws.name}\n"
      f"Resource Group     : {ws.resource_group}\n"
      f"Location           : {ws.location}\n"
      f"Subscription ID    : {ws.subscription_id}\n")

Azure Machine Learning Workspace Loaded Successfully
-----------------------------------------------------------
Workspace Name     : optimize-machine-learning-ws
Resource Group     : optimize-machine-learning
Location           : norwayeast
Subscription ID    : 48778e11-0fc7-4fc8-a16c-304a430e61a4



### Initialize Experiment

In [2]:
from azureml.core import Experiment

# Define experiment name and initialize it in the workspace
experiment_name = "bank-deposit-predictor"
experiment = Experiment(workspace=ws, name=experiment_name)

# Define project folder
project_folder = "./bank-deposit-predictor"

# Display experiment details
print(f"Azure ML Experiment Initialized Successfully\n"
      f"--------------------------------------------------\n"
      f"Experiment Name    : {experiment.name}\n"
      f"Workspace          : {experiment.workspace.name}\n"
      f"Project Folder     : {project_folder}\n")


Azure ML Experiment Initialized Successfully
--------------------------------------------------
Experiment Name    : bank-deposit-predictor
Workspace          : optimize-machine-learning-ws
Project Folder     : ./bank-deposit-predictor



### Handling the Compute Cluster

In [3]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# -------------------------------
# Compute Cluster Configuration
# -------------------------------
compute_name = "training-cluster"
compute_min_nodes = 0
compute_max_nodes = 5
vm_size = "Standard_D3_v2"

# Define provisioning configuration for the compute cluster
compute_config = AmlCompute.provisioning_configuration(
    vm_size=vm_size,
    min_nodes=compute_min_nodes,
    max_nodes=compute_max_nodes,
    idle_seconds_before_scaledown=600,  # Auto-scale down after 10 minutes of inactivity
)

# -------------------------------
# Create or Attach Compute Cluster
# -------------------------------
try:
    # Check if the compute cluster already exists in the workspace
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
    print(f"✔️ Compute cluster '{compute_name}' found in the workspace.")
    print(f"   - VM Size         : {compute_target.vm_size}")
    print(f"   - Minimum Nodes   : {compute_target.scale_settings.minimum_node_count}")
    print(f"   - Maximum Nodes   : {compute_target.scale_settings.maximum_node_count}")
    print(f"   - Auto-Scale Down : {compute_target.scale_settings.idle_seconds_before_scaledown // 60} minutes")
except ComputeTargetException:
    # Create a new compute cluster if it does not exist
    print(f"🚀 Creating a new compute cluster: '{compute_name}'")
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    compute_target.wait_for_completion(show_output=True)
    print(f"✅ Compute cluster '{compute_name}' has been successfully created.")

print(f"🔹 Compute cluster '{compute_name}' is ready for use.")



🚀 Creating a new compute cluster: 'training-cluster'
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
✅ Compute cluster 'training-cluster' has been successfully created.
🔹 Compute cluster 'training-cluster' is ready for use.


### Importing the Dataset

In [4]:
from azureml.core import Dataset

# -------------------------------
# Dataset Configuration
# -------------------------------
dataset_name = "bank-marketing-dataset"
dataset_description = "Bank Marketing Dataset for predictive modeling."
dataset_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

# -------------------------------
# Check if Dataset Exists
# -------------------------------
if dataset_name in ws.datasets:
    dataset = ws.datasets[dataset_name]
    print(f"✔️ Found existing dataset: '{dataset_name}' in the workspace.")
else:
    print(f"📂 Dataset '{dataset_name}' not found. Registering a new dataset...")
    
    # Create AML Tabular Dataset from URL
    dataset = Dataset.Tabular.from_delimited_files(dataset_url)
    
    # Register the dataset in the workspace
    dataset = dataset.register(
        workspace=ws,
        name=dataset_name,
        description=dataset_description
    )
    
    print(f"✅ Dataset '{dataset_name}' has been successfully registered.")

# -------------------------------
# Load Dataset into DataFrame
# -------------------------------
df = dataset.to_pandas_dataframe()
print(f"📊 Dataset '{dataset_name}' loaded successfully with {df.shape[0]} rows and {df.shape[1]} columns.")
df.head()


✔️ Found existing dataset: 'bank-marketing-dataset' in the workspace.
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
📊 Dataset 'bank-marketing-dataset' loaded successfully with 32950 rows and 21 columns.


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,57,technician,married,high.school,no,no,yes,cellular,may,mon,...,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1,no
1,55,unknown,married,unknown,unknown,yes,no,telephone,may,thu,...,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0,no
2,33,blue-collar,married,basic.9y,no,no,no,cellular,may,fri,...,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1,no
3,36,admin.,married,high.school,no,no,no,telephone,jun,fri,...,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1,no
4,27,housemaid,married,high.school,no,yes,no,cellular,jul,fri,...,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1,no


### Retrieving Summary Statistics on Data  

In [5]:
# Generate summary statistics for numerical columns
summary_stats = df.describe().T[['mean', 'std', 'min', '25%', '50%', '75%', 'max']]

# Format numbers to two decimal places
summary_stats = summary_stats.round(2)

# Display the enhanced summary statistics table
print("\n📊 Summary Statistics Table")
print("=" * 80)
print(summary_stats.to_string())




📊 Summary Statistics Table
                   mean     std      min      25%      50%      75%      max
age               40.04   10.43    17.00    32.00    38.00    47.00    98.00
duration         257.34  257.33     0.00   102.00   179.00   318.00  4918.00
campaign           2.56    2.76     1.00     1.00     2.00     3.00    56.00
pdays            962.17  187.65     0.00   999.00   999.00   999.00   999.00
previous           0.17    0.50     0.00     0.00     0.00     0.00     7.00
emp.var.rate       0.08    1.57    -3.40    -1.80     1.10     1.40     1.40
cons.price.idx    93.57    0.58    92.20    93.08    93.75    93.99    94.77
cons.conf.idx    -40.52    4.62   -50.80   -42.70   -41.80   -36.40   -26.90
euribor3m          3.62    1.74     0.63     1.34     4.86     4.96     5.04
nr.employed     5166.86   72.21  4963.60  5099.10  5191.00  5228.10  5228.10


### Setting up Training Configuration

In [6]:
from azureml.train.automl import AutoMLConfig

# Define AutoML settings with updated parameters
automl_settings = {
    "experiment_timeout_minutes": 20,  # 20 minutes
    "max_concurrent_iterations": 5,  # Number of concurrent iterations
    "primary_metric": "accuracy",  # Classification typically optimizes for accuracy
    "n_cross_validations": 5  # Maintain robust evaluation
}

# Configure AutoML for classification with model explainability
classification_config = AutoMLConfig(
    compute_target=compute_target,
    task="classification",
    training_data=dataset,
    label_column_name="y",
    path=project_folder,
    enable_early_stopping=True,
    featurization="auto",
    debug_log="automl_errors.log",
    **automl_settings
)


### Setting Up Information Containers

In [7]:
from azureml.pipeline.core import PipelineData, TrainingOutput

# Retrieve the default datastore
default_datastore = ws.get_default_datastore()

# Define pipeline output identifiers
METRICS_OUTPUT_NAME = "metrics_output"
MODEL_OUTPUT_NAME = "best_model_output"

# Define PipelineData for storing training metrics
metrics_data = PipelineData(
    name="metrics_data",
    datastore=default_datastore,
    pipeline_output_name=METRICS_OUTPUT_NAME,
    training_output=TrainingOutput(type="Metrics")  # Stores training metrics
)

# Define PipelineData for storing the best model
model_data = PipelineData(
    name="model_data",
    datastore=default_datastore,
    pipeline_output_name=MODEL_OUTPUT_NAME,
    training_output=TrainingOutput(type="Model")  # Stores trained model
)

# Confirmation message
print("✅ Pipeline outputs initialized successfully.")
print(f"📊 Metrics Output: {METRICS_OUTPUT_NAME}")
print(f"📦 Model Output: {MODEL_OUTPUT_NAME}")


✅ Pipeline outputs initialized successfully.
📊 Metrics Output: metrics_output
📦 Model Output: best_model_output


### Creating the AutoML Pipeline

In [8]:
from azureml.pipeline.steps import AutoMLStep
from azureml.pipeline.core import Pipeline

# Define the AutoML step
automl_step = AutoMLStep(
    name="AutoML_Classification",
    automl_config=classification_config,
    outputs=[metrics_data, model_data],  # Capture training metrics and model output
    allow_reuse=True  # Enables step reuse to optimize execution time
)

# Construct the pipeline with AutoML step
pipeline = Pipeline(
    workspace=ws,
    steps=[automl_step],
    description="Automated ML pipeline for classiying banking conversion rates"
)

# Confirmation message
print("✅ AutoML pipeline successfully created.")


✅ AutoML pipeline successfully created.


## Traning the Data

### Submit the AutoML Job

In [9]:
from azureml.core import Experiment

# Submit the pipeline experiment
try:
    run = experiment.submit(pipeline)
    print(f"✅ Experiment \"{experiment_name}\" successfully submitted. Run ID: {run.id}")

    # Monitor run progress with live logs
    print("🔄 Waiting for pipeline execution to complete...")
    run.wait_for_completion(show_output=True)
    
    # Retrieve and display run details
    run_details = run.get_details()
    print("\nRun Details:")
    for key, value in run_details.items():
        print(f"{key}: {value}")

except Exception as e:
    print(f"⚠️ Error: {str(e)}")


Created step AutoML_Classification [4e3a7fd3][d9051a53-df2c-4c36-b87f-c7c0de63df41], (This step will run and generate new outputs)
Submitted PipelineRun 48387dc9-3b6c-49ae-a0ef-26b9b3389d1b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/48387dc9-3b6c-49ae-a0ef-26b9b3389d1b?wsid=/subscriptions/48778e11-0fc7-4fc8-a16c-304a430e61a4/resourcegroups/optimize-machine-learning/workspaces/optimize-machine-learning-ws&tid=f804f881-90d8-4e7d-8309-7fb565eaaf2c
✅ Experiment "bank-deposit-predictor" successfully submitted. Run ID: 48387dc9-3b6c-49ae-a0ef-26b9b3389d1b
🔄 Waiting for pipeline execution to complete...
PipelineRunId: 48387dc9-3b6c-49ae-a0ef-26b9b3389d1b
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/48387dc9-3b6c-49ae-a0ef-26b9b3389d1b?wsid=/subscriptions/48778e11-0fc7-4fc8-a16c-304a430e61a4/resourcegroups/optimize-machine-learning/workspaces/optimize-machine-learning-ws&tid=f804f881-90d8-4e7d-8309-7fb565eaaf2c
PipelineRun Status: NotStarted
PipelineRu

## Viewing the Results

### Retrieving the Metrics of All Child Runs

In [10]:
import json
import pandas as pd

# Retrieve the pipeline output for metrics
try:
    metrics_output = run.get_pipeline_output(METRICS_OUTPUT_NAME)  # Use `run` instead of `pipeline_run`
    num_files_downloaded = metrics_output.download('.', show_progress=True)

    print(f"✅ Successfully downloaded {num_files_downloaded} file(s) from pipeline output.")

    # Load and deserialize metrics data
    with open(metrics_output._path_on_datastore, "r", encoding="utf-8") as f:
        metrics_output_result = f.read()

    deserialized_metrics_output = json.loads(metrics_output_result)
    df = pd.DataFrame(deserialized_metrics_output)

    # Display the DataFrame
    print(df)

except Exception as e:
    print(f"⚠️ Error retrieving pipeline output: {str(e)}")


Downloaded azureml/5039813b-7cb8-4728-b150-780f33c68dd8/metrics_data, 1 files out of an estimated total of 1
✅ Successfully downloaded 1 file(s) from pipeline output.
                                 5039813b-7cb8-4728-b150-780f33c68dd8_2  \
precision_score_macro                              [0.6034149496984739]   
precision_score_weighted                           [0.8739530478570771]   
average_precision_score_micro                      [0.8171304713489891]   
recall_score_micro                                 [0.7355690440060698]   
average_precision_score_macro                       [0.711454441290039]   
accuracy                                           [0.7355690440060698]   
norm_macro_recall                                  [0.4444921276299585]   
precision_score_micro                              [0.7355690440060698]   
weighted_accuracy                                  [0.7388861489613122]   
f1_score_macro                                     [0.6032562548115933]   
recall_s

### Retrieve the Best Model

In [11]:
import pickle

# Retrieve the best model from the pipeline run
try:
    best_model_output = run.get_pipeline_output(MODEL_OUTPUT_NAME)
    num_files_downloaded = best_model_output.download('.', show_progress=True)

    print(f"✅ Successfully downloaded {num_files_downloaded} file(s) from the pipeline output.")

    # Load the model from the downloaded file
    model_path = best_model_output._path_on_datastore

    if not model_path:
        raise ValueError("⚠️ Error: Model output file path is empty or invalid.")

    with open(model_path, "rb") as f:
        best_model = pickle.load(f)

    print("✅ Model successfully loaded.")

    # Display model steps (if applicable)
    if hasattr(best_model, "steps"):
        print("🔹 Model Steps:")
        for step in best_model.steps:
            print(f"  - {step}")
    else:
        print("⚠️ Warning: The loaded model does not have a `steps` attribute.")

except Exception as e:
    print(f"⚠️ Error retrieving or loading the model: {str(e)}")


Downloaded azureml/5039813b-7cb8-4728-b150-780f33c68dd8/model_data, 1 files out of an estimated total of 1
✅ Successfully downloaded 1 file(s) from the pipeline output.
✅ Model successfully loaded.
🔹 Model Steps:
  - ('datatransformer', DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification'))
  - ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('3', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l2')), ('xgboostclassifier', XGBoo

## Setting Up the the RestAPI for the Pipeline

### Publish the Pipeline

In [12]:
# Publish the pipeline with metadata
published_pipeline = pipeline.publish(
    name="Bank Marketing Predictor Pipeline",
    description="This pipeline is used to train an ML algorithm to recommend a bank savings account.",
    version="1.0"
)

print(f"✅ Pipeline successfully published. Pipeline ID: {published_pipeline.id}")

✅ Pipeline successfully published. Pipeline ID: 97cc9a29-90fe-4090-af3d-1fdb463da669


### Authenticate and Retrieve the Authorization Header

In [13]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()


### Trigger the Pipeline Execution via REST API

In [14]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(
    rest_endpoint,
    headers=auth_header,
    json={"ExperimentName": "pipeline-rest-endpoint"}
)


### Handle API Response and Extract Run ID

In [15]:
try:
    response.raise_for_status()
except Exception:
    raise Exception(
        "Received an error response from the endpoint:\n"
        f"🔹 Endpoint: {rest_endpoint}\n"
        f"🔹 Response Code: {response.status_code}\n"
        f"🔹 Headers: {response.headers}\n"
        f"🔹 Content: {response.content}"
    )

run_id = response.json().get('Id')
print(f"✅ Pipeline run successfully submitted. Run ID: {run_id}")

✅ Pipeline run successfully submitted. Run ID: 58b329f2-215c-4bf7-bf69-614058b0686a


### Monitor the Pipeline Execution

In [16]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["pipeline-rest-endpoint"], run_id)
RunDetails(published_pipeline_run).show()


2025-03-11 12:48:06.448628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-11 12:48:07.428270: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-11 12:48:07.723000: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-11 12:48:09.922346: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 12:48:15.037404: W tensorflow/compiler/tf2

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …

ERROR:azureml.data._dataset_client:[NOT_SUPPORTED_API_USE_ATTEMPT] The [_DatasetClient.get] API has been deprecated and is no longer supported
